# In this notebook, I use autoencoder for this fraud detection. The fraud patterns are learnt from the given hosts that are known to receive substantial amounts of fraudulent traffic

For some context, when an ad is shown on a webpage you are visiting, the image (or
animation or video) is loaded into your browser from another site (ad server) and, along
with the request for the ad your browser may also generate requests to other servers,
e.g. the ones of the tracking companies who want to learn about your habits or the
companies which measure where the ads are shown, for how long they are seen, etc.
This data was collected by one of such measurement companies.

Advertising fraud is a huge problem for digital media and brands. You will be trying to
help solve this problem. Attached is a file with one day of ad impression logs for selected IP addresses. The fields are, in order:
- Timestamp
- IP address
- Detected browser type
- User agent string
- Host (URL)
- Whether the impression was in view (1.0 = yes, 0.0 = no)
- Number of plugins installed
- Browser window position and size (x, y, width, height)
- Network latency

Your task is to identify hosts which are receiving a substantial amount of fraudulent
traffic. As part of this, you may also wish to identify IP addresses home to machines that
are part of botnets, but this is not required. The definition of "substantial" is up to you --
this may be a ranked list of all hosts, or a list of hosts reaching a certain threshold, or
you may choose not to quantify the amount of fraud and simply classify hosts as likely
to be experiencing high fraud or not.
To get you started, here is a list of hosts which are known to receive substantial
amounts of fraudulent traffic:

- featureplay.com
- uvidi.com
- spryliving.com
- greatboxgames.com
- mmabay.co.uk
- workingmothertv.com
- besthorrorgame.com
- dailyparent.com
- superior-movies.com
- yourhousedesign.com
- outdoorlife.tv
- drumclub.info
- cycleworld.tv
- hmnp.us
- nlinevideos.com

In [ ]:
import pandas as pd
import numpy as np

## Load the data, profile the data

In [ ]:
data = 'HW_assignment_data_set.tsv'
data_df = pd.read_csv(data, names=['timestamp', 'ip', 'browser_type', 'user_agent_string', 'host', 'viewed',
                                   'no_of_plugins', 'position', 'network_latency'],
                      sep='\t', parse_dates=['timestamp'])
data_df.shape

(235083, 9)

In [ ]:
data_df.head()

,timestamp,ip,browser_type,user_agent_string,host,viewed,no_of_plugins,position,network_latency
0,2014-08-25,393.414.443.469,Safari/Webkit,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,http://www.domain.com.au,0.0,NaN,"(0,0,1280,629)",0.0
1,2014-08-25,393.414.443.469,Safari/Webkit,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,http://www.domain.com.au,0.0,NaN,"(0,0,1280,629)",0.0
2,2014-08-25,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,http://www.mangareader.net,NaN,NaN,NaN,NaN
3,2014-08-25,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,http://www.mangareader.net,NaN,NaN,NaN,NaN
4,2014-08-25,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,http://www.mangareader.net,NaN,NaN,NaN,NaN


In [ ]:
data_df.dtypes

timestamp            datetime64[ns]
ip                           object
browser_type                 object
user_agent_string            object
host                         object
viewed                      float64
no_of_plugins               float64
position                     object
network_latency             float64
dtype: object

In [ ]:
data_df.describe(include='all')

,timestamp,ip,browser_type,user_agent_string,host,viewed,no_of_plugins,position,network_latency
count,235083,235083,235083,235075,235083,211811.000000,162304.000000,118451,2.002830e+05
unique,74914,8216,6,4778,11779,NaN,NaN,10302,NaN
top,2014-08-25 23:18:24,411.517.507.552,Chrome,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,http://pixel.adsafeprotected.com,NaN,NaN,"(0,0,1366,667)",NaN
freq,20,12828,76595,26509,20548,NaN,NaN,2271,NaN
first,2014-08-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2014-08-25 23:59:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,0.431998,10.139442,NaN,3.452002e+02
std,NaN,NaN,NaN,NaN,NaN,0.495355,7.637172,NaN,1.329282e+04
min,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,NaN,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,0.000000,2.000000,NaN,0.000000e+00


## Feature engineering: One hot encoder on brower type

In [ ]:
data_df = pd.concat([data_df, pd.get_dummies(data_df['browser_type'])], axis=1)
data_df.head()

,timestamp,ip,browser_type,user_agent_string,host,viewed,no_of_plugins,position,network_latency,Chrome,Firefox,Internet Explorer,Opera,Safari/Webkit,Unknown
0,2014-08-25,393.414.443.469,Safari/Webkit,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,http://www.domain.com.au,0.0,NaN,"(0,0,1280,629)",0.0,0,0,0,0,1,0
1,2014-08-25,393.414.443.469,Safari/Webkit,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,http://www.domain.com.au,0.0,NaN,"(0,0,1280,629)",0.0,0,0,0,0,1,0
2,2014-08-25,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,http://www.mangareader.net,NaN,NaN,NaN,NaN,0,0,0,0,0,1
3,2014-08-25,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,http://www.mangareader.net,NaN,NaN,NaN,NaN,0,0,0,0,0,1
4,2014-08-25,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,http://www.mangareader.net,NaN,NaN,NaN,NaN,0,0,0,0,0,1


## Feature engineering: Calculate browser area

In [ ]:
def browser_area_cal(s):
    if s != s:
        return s
    else:
        _, _, width, height = s.split(',')
        width = int(width)
        height = int(height[:-1])
        return width*height

In [ ]:
data_df['browser_area'] = data_df['position'].apply(browser_area_cal)
data_df.head()

,timestamp,ip,browser_type,user_agent_string,host,viewed,no_of_plugins,position,network_latency,Chrome,Firefox,Internet Explorer,Opera,Safari/Webkit,Unknown,browser_area
0,2014-08-25,393.414.443.469,Safari/Webkit,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,http://www.domain.com.au,0.0,NaN,"(0,0,1280,629)",0.0,0,0,0,0,1,0,805120.0
1,2014-08-25,393.414.443.469,Safari/Webkit,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8)...,http://www.domain.com.au,0.0,NaN,"(0,0,1280,629)",0.0,0,0,0,0,1,0,805120.0
2,2014-08-25,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,http://www.mangareader.net,NaN,NaN,NaN,NaN,0,0,0,0,0,1,NaN
3,2014-08-25,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,http://www.mangareader.net,NaN,NaN,NaN,NaN,0,0,0,0,0,1,NaN
4,2014-08-25,325.441.386.395,Unknown,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,http://www.mangareader.net,NaN,NaN,NaN,NaN,0,0,0,0,0,1,NaN


## Generate train and test data based on a list of hosts which are known to receive substantial amounts of fraudulent traffic

In [ ]:
host_list = ['featureplay.com', 'uvidi.com', 'spryliving.com', 'greatboxgames.com', 'mmabay.co.uk',
             'workingmothertv.com', 'besthorrorgame.com', 'dailyparent.com','superior-movies.com',
             'yourhousedesign.com', 'outdoorlife.tv', 'drumclub.info', 'cycleworld.tv', 'hmnp.us', 'nlinevideos.com']

In [ ]:
# train_df: known hosts
train_df = data_df[data_df['host'].apply(lambda full_host: any(host in full_host for host in host_list))]
train_df.shape

(2343, 16)

In [ ]:
# test_df: unknown hosts
test_df = data_df[~data_df['host'].apply(lambda full_host: any(host in full_host for host in host_list))]
test_df.shape

(232740, 16)

In [ ]:
train_df.shape[0] + test_df.shape[0] - data_df.shape[0]

0

## Botnets IPs: Top IPs by count in train data

In [ ]:
pd.DataFrame(train_df['ip'].value_counts())

,ip
324.338.423.496,224
496.529.325.519,219
574.491.567.341,208
476.494.399.426,186
529.366.487.475,156
496.437.522.387,139
574.452.484.501,136
496.325.356.347,76
489.462.542.447,67
438.562.383.508,66


## Build an autoencoder on train_data to learn the hosts who has received substantial amounts of fraudulent traffic

#### For simple, we only use features ['viewed', 'no_of_plugins', 'network_latency', 'Chrome', 'Firefox', 'Internet Explorer', 'Opera', 'Safari/Webkit', 'Unknown', 'browser_area'] 

In [ ]:
selected_features = ['viewed', 'no_of_plugins', 'network_latency', 'Chrome', 'Firefox', 'Internet Explorer', 'Opera',
                     'Safari/Webkit', 'Unknown', 'browser_area']

In [ ]:
train_selected = train_df[selected_features]
test_selected = test_df[selected_features]
train_selected.describe(include='all')

,viewed,no_of_plugins,network_latency,Chrome,Firefox,Internet Explorer,Opera,Safari/Webkit,Unknown,browser_area
count,2265.000000,1559.000000,2074.000000,2343.000000,2343.000000,2343.000000,2343.0,2343.000000,2343.000000,2.012000e+03
mean,0.776600,5.780629,2372.299421,0.428937,0.068715,0.466069,0.0,0.005548,0.030730,8.672783e+05
std,0.416616,8.048353,8356.602506,0.495030,0.253023,0.498954,0.0,0.074297,0.172622,4.170205e+05
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00
25%,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,6.891520e+05
50%,1.000000,3.000000,127.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,7.239680e+05
75%,1.000000,3.000000,942.500000,1.000000,0.000000,1.000000,0.0,0.000000,0.000000,1.052898e+06
max,1.000000,26.000000,221849.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,4.096000e+06


#### We first fill np.nan by mean values

In [ ]:
train_selected = train_selected.fillna(train_selected.mean())
test_selected = test_selected.fillna(test_selected.mean())
train_selected.describe(include='all')

,viewed,no_of_plugins,network_latency,Chrome,Firefox,Internet Explorer,Opera,Safari/Webkit,Unknown,browser_area
count,2343.00000,2343.000000,2343.000000,2343.000000,2343.000000,2343.000000,2343.0,2343.000000,2343.000000,2.343000e+03
mean,0.77660,5.780629,2372.299421,0.428937,0.068715,0.466069,0.0,0.005548,0.030730,8.672783e+05
std,0.40962,6.564434,7862.052698,0.495030,0.253023,0.498954,0.0,0.074297,0.172622,3.864293e+05
min,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00
25%,1.00000,2.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,6.891520e+05
50%,1.00000,3.000000,155.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,8.672783e+05
75%,1.00000,5.780629,2372.299421,1.000000,0.000000,1.000000,0.0,0.000000,0.000000,1.049088e+06
max,1.00000,26.000000,221849.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,4.096000e+06


#### We normalize three features to the range [0, 1]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
min_max_scaler = MinMaxScaler()

In [ ]:
X_train_3features = min_max_scaler.fit_transform(train_selected[['no_of_plugins', 'network_latency', 'browser_area']])
X_train_3features

array([[0.19122514, 0.01575847, 0.16825   ],
       [0.08      , 0.        , 0.1765    ],
       [0.19122514, 0.00064909, 0.32      ],
       ...,
       [0.19122514, 0.0011314 , 0.22444287],
       [0.19122514, 0.01069331, 0.22444287],
       [0.04      , 0.        , 0.32      ]])

In [ ]:
X_test_3features = min_max_scaler.transform(test_selected[['no_of_plugins', 'network_latency', 'browser_area']])
X_test_3features

array([[3.67268655e-01, 0.00000000e+00, 1.96562500e-01],
       [3.67268655e-01, 0.00000000e+00, 1.96562500e-01],
       [3.67268655e-01, 1.46040423e-03, 9.35165208e+06],
       ...,
       [3.67268655e-01, 1.46040423e-03, 9.35165208e+06],
       [3.20000000e-01, 7.66286979e-04, 9.35165208e+06],
       [0.00000000e+00, 0.00000000e+00, 2.90625000e-02]])

In [ ]:
rest_features = [feature for feature in selected_features 
                 if feature not in ['no_of_plugins', 'network_latency', 'browser_area']]
rest_features

['viewed',
 'Chrome',
 'Firefox',
 'Internet Explorer',
 'Opera',
 'Safari/Webkit',
 'Unknown']

In [ ]:
X_train = np.concatenate((X_train_3features, train_selected[rest_features].values), axis=1)
X_test = np.concatenate((X_test_3features, test_selected[rest_features].values), axis=1)
X_train

array([[0.19122514, 0.01575847, 0.16825   , ..., 0.        , 0.        ,
        0.        ],
       [0.08      , 0.        , 0.1765    , ..., 0.        , 0.        ,
        0.        ],
       [0.19122514, 0.00064909, 0.32      , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.19122514, 0.0011314 , 0.22444287, ..., 0.        , 0.        ,
        0.        ],
       [0.19122514, 0.01069331, 0.22444287, ..., 0.        , 0.        ,
        0.        ],
       [0.04      , 0.        , 0.32      , ..., 0.        , 0.        ,
        0.        ]])

## Build Autoencoder on X_train for fraud detection, including encoder and decoder

In [ ]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model

Using TensorFlow backend.


In [ ]:
input_dim = X_train.shape[1]
encoding_dim = input_dim//2
print(input_dim, encoding_dim)

10 5


In [ ]:
# Input placeholder, dropout rate, encorder and decorder
input_att = Input(shape=(input_dim,))
input_dropout = Dropout(0.2)(input_att)
encoded = Dense(encoding_dim, activation='relu')(input_dropout)
decoded = Dense(input_dim, activation='linear')(encoded)

In [ ]:
autoencoder = Model(input_att, decoded)
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
Total params: 115
Trainable params: 115
Non-trainable params: 0
_________________________________________________________________


In [ ]:
autoencoder.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
autoencoder.fit(X_train, X_train, epochs=100, shuffle=True, validation_split=0.2, verbose=1)

Train on 1874 samples, validate on 469 samples
Epoch 1/100
1874/1874 [==============================] - 0s 248us/step - loss: 0.2024 - val_loss: 0.1476
Epoch 2/100
1874/1874 [==============================] - 0s 74us/step - loss: 0.1344 - val_loss: 0.1082
Epoch 3/100
1874/1874 [==============================] - 0s 75us/step - loss: 0.1029 - val_loss: 0.0898
Epoch 4/100
1874/1874 [==============================] - 0s 70us/step - loss: 0.0880 - val_loss: 0.0802
Epoch 5/100
1874/1874 [==============================] - 0s 63us/step - loss: 0.0792 - val_loss: 0.0748
Epoch 6/100
1874/1874 [==============================] - 0s 66us/step - loss: 0.0727 - val_loss: 0.0712
Epoch 7/100
1874/1874 [==============================] - 0s 79us/step - loss: 0.0697 - val_loss: 0.0679
Epoch 8/100
1874/1874 [==============================] - 0s 65us/step - loss: 0.0667 - val_loss: 0.0648
Epoch 9/100
1874/1874 [==============================] - 0s 66us/step - loss: 0.0634 - val_loss: 0.0614
Epoch 10/100
187

1874/1874 [==============================] - 0s 83us/step - loss: 0.0288 - val_loss: 0.0266
Epoch 79/100
1874/1874 [==============================] - 0s 110us/step - loss: 0.0294 - val_loss: 0.0267
Epoch 80/100
1874/1874 [==============================] - 0s 71us/step - loss: 0.0306 - val_loss: 0.0271
Epoch 81/100
1874/1874 [==============================] - 0s 73us/step - loss: 0.0288 - val_loss: 0.0267
Epoch 82/100
1874/1874 [==============================] - 0s 71us/step - loss: 0.0297 - val_loss: 0.0267
Epoch 83/100
1874/1874 [==============================] - 0s 70us/step - loss: 0.0302 - val_loss: 0.0265
Epoch 84/100
1874/1874 [==============================] - 0s 67us/step - loss: 0.0292 - val_loss: 0.0265
Epoch 85/100
1874/1874 [==============================] - 0s 87us/step - loss: 0.0279 - val_loss: 0.0268
Epoch 86/100
1874/1874 [==============================] - 0s 82us/step - loss: 0.0284 - val_loss: 0.0266
Epoch 87/100
1874/1874 [==============================] - 0s 79us/s

#### Evaluate the loss on X_ train and X_test: huge difference

In [ ]:
autoencoder.evaluate(X_train, X_train)

2343/2343 [==============================] - 0s 48us/step


0.019190713044373628

In [ ]:
autoencoder.evaluate(X_test, X_test)

232740/232740 [==============================] - 8s 35us/step


2.842567490605635e+17

## Define function to calculate mse for each record

In [ ]:
def mse_for_each_record(act, pred):
    error = np.square(act - pred)
    squared_error = np.square(error)
    mean_squared_error = np.mean(squared_error, axis=1)
    return mean_squared_error

In [ ]:
pred_train = autoencoder.predict(X_train)
pred_test = autoencoder.predict(X_test)

In [ ]:
mse_train = mse_for_each_record(X_train, pred_train)
mse_test = mse_for_each_record(X_test, pred_test)

In [ ]:
print("-------mse_train-------")
print(pd.Series(mse_train).describe())
print("\n-------mse_test-------")
print(pd.Series(mse_test).describe())

-------mse_train-------
count    2343.000000
mean        0.007036
std         0.019955
min         0.000024
25%         0.000070
50%         0.000127
75%         0.002992
max         0.111754
dtype: float64

-------mse_test-------
count    2.327400e+05
mean     6.757804e+40
std      2.305288e+43
min      4.164897e-05
25%      7.424011e-03
50%      1.896629e-01
75%      6.860657e+26
max      7.864056e+45
dtype: float64


## Decide the fraud threshold as the 99% percentile of train data

In [ ]:
threshold = np.percentile(mse_train,99)

print("Fraud-Threshold = {}".format(threshold))

Fraud-Threshold = 0.10048884204631418


In [ ]:
print("Fraud records in test data = {}%".format(np.sum(mse_test <= threshold)/X_test.shape[0]*100))

Fraud records in test data = 39.75251353441608%


In [ ]:
fraud_in_test = test_df[mse_test <= threshold]
fraud_in_test.shape

(92520, 16)

In [ ]:
fraud_in_test.head()

,timestamp,ip,browser_type,user_agent_string,host,viewed,no_of_plugins,position,network_latency,Chrome,Firefox,Internet Explorer,Opera,Safari/Webkit,Unknown,browser_area
5,2014-08-25 00:00:00,326.432.563.561,Chrome,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,http://failblog.cheezburger.com,0.0,25.0,"(0,0,1366,643)",73.0,1,0,0,0,0,0,878338.0
10,2014-08-25 00:00:04,369.438.566.432,Chrome,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,http://www.stuffyoushouldknow.com,0.0,17.0,"(0,0,1920,993)",0.0,1,0,0,0,0,0,1906560.0
11,2014-08-25 00:00:04,369.438.566.432,Chrome,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,http://www.stuffyoushouldknow.com,0.0,17.0,"(0,0,1920,993)",0.0,1,0,0,0,0,0,1906560.0
15,2014-08-25 00:00:06,488.432.432.344,Chrome,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,http://madamenoire.com,0.0,23.0,"(0,0,1366,643)",139.0,1,0,0,0,0,0,878338.0
22,2014-08-25 00:00:09,325.554.366.399,Chrome,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,http://jobsearch.about.com,1.0,16.0,"(0,0,1366,643)",0.0,1,0,0,0,0,0,878338.0


## Autoencoder suggests those hosts which receive substantial amounts of fraudulent traffic

In [ ]:
# Top hosts by count are my results:
fraud_in_test['host'].value_counts()

http://pixel.adsafeprotected.com            2202
http://www.answers.com                      1970
http://www.domain.com.au                    1839
http://www.mynet.com                        1692
http://fw.adsafeprotected.com               1642
http://www.stuffyoushouldknow.com           1522
http://www.pandora.com                      1452
http://www.cars.com                         1419
http://www.amazon.com                       1026
http://www.startribune.com                   837
http://www.huffingtonpost.com                834
http://www.usmagazine.com                    681
http://living.msn.com                        671
http://www.ebay.co.uk                        622
http://celebs.answers.com                    607
http://www.dailymail.co.uk                   546
http://webmaila.juno.com                     532
http://www.realtor.com                       524
http://www.ratemyprofessors.com              517
http://www.cnn.com                           514
http://www.pch.com  

## Autoencoder suggests botnets IPs by IP count. Noticing that IP like 529.366.487.475 shows in both train and test data

In [ ]:
# Top IP by count are suggested to be botnets.
pd.DataFrame(fraud_in_test['ip'].value_counts())

,ip
411.517.507.552,6785
358.472.462.434,2524
337.508.436.325,1982
411.374.349.559,1857
369.438.566.432,1528
544.531.344.383,1293
391.367.519.448,1267
357.394.348.456,1008
529.366.487.475,967
515.446.450.448,921


# If I have more time:
- I will apply feature engineering on the feature 'user_agent_string' and extract key tokens from it.